Este caderno trata-se de um exercício da disciplina de Aprendizado de Máquina lecionada na Universidade Federal do Rio Grande do Norte (UFRN). A intenção é fazermos o pré-processamento de 800 imagens de cachorros e gatos, e classificarmos essas imagens em diversos métodos, simulando como é o modus operandi quando estamos trabalhando com um problema de machine learning.

**Nota:** Ao todo, utilizaremos de 12 base de dados provenientes do pré-processamento das imagens, onde a primeira coluna será a classe:
* $1=cachorro$
* $0=gato$

# **MÉTODO HOG DE TRANSFORMAÇÃO**
---
---

Para o método HOG teremos 4 dataset, utilizando dos presets:
* tamanho: (128,128) e (256,256)
* pixels: (16,16) e (20,20)

## **1. Importando as Bibliotecas**

In [7]:
import glob
from skimage import io, color
from skimage.transform import resize
from skimage.feature import hog
import pandas as pd
import os
from tqdm import tqdm


## **2.Importando o Dataset**

## **3.Implantando o HOG nas Imagens**

In [8]:
%%time
# Diretório onde estão as imagens
caminho_das_imagens = './imagens/*.jpg'

# Parâmetros HOG
tamanhos = [(128, 128), (256, 256)]
pixels_per_cell_options = [(16, 16), (20, 20)]

# Dicionário para armazenar DataFrames dinamicamente
dfs = {}

# Itera sobre diferentes tamanhos e pixels_per_cell
for size in tamanhos:
    for cell in pixels_per_cell_options:
        # Lista para armazenar as features HOG e labels
        data = []

        # Itera sobre todas as imagens no diretório
        for file_path in sorted(glob.glob(caminho_das_imagens)):
            # Nome do arquivo da imagem
            nome_imagem = os.path.basename(file_path)

            # Rótulo: 1 se a primeira letra do nome do arquivo é minúscula, 0 caso contrário
            label = 1 if nome_imagem[0].islower() else 0

            # Lê a imagem
            imagem = io.imread(file_path)

            # Redimensiona a imagem
            imagem_reduzida = resize(imagem, size)

            # Calcula o HOG
            fd = hog(imagem_reduzida, orientations=9, pixels_per_cell=cell,
                     cells_per_block=(2, 2), block_norm='L2-Hys', channel_axis=-1)

            # Adiciona o label e as features em uma lista
            data.append([label] + fd.tolist())

        # Cria o DataFrame com o label e as features
        colunas = ['label'] + [f'feature_{i}' for i in range(len(fd))]
        df_name = f'hog_{size[0]}_{cell[0]}'  # Nome do DataFrame
        dfs[df_name] = pd.DataFrame(data, columns=colunas)


CPU times: total: 50.6 s
Wall time: 1min 5s


### 3.1.Conferindo o `.shape()` dos datasets gerados

In [9]:
dfs['hog_128_16'].shape, dfs['hog_128_20'].shape, dfs['hog_256_16'].shape, dfs['hog_256_20'].shape



((798, 1765), (798, 901), (798, 8101), (798, 4357))

### 3.2.Salvando em CSV

In [ ]:
to_csv(os.path.join(diretorio_saida, f'{nome_df}.csv'), index=False)

In [10]:
# Diretório onde os arquivos CSV serão salvos
diretorio_saida = './datasets'
os.makedirs(diretorio_saida, exist_ok=True)

# Salvar cada DataFrame como CSV
for nome_df, df in dfs.items():
    caminho_csv = os.path.join(diretorio_saida, f'{nome_df}.csv')
    df.to_csv(caminho_csv, index=False)
    print(f'Salvou {caminho_csv}')

Salvou ./datasets\hog_128_16.csv
Salvou ./datasets\hog_128_20.csv
Salvou ./datasets\hog_256_16.csv
Salvou ./datasets\hog_256_20.csv


# **MÉTODO CNN DE TRANSFORMAÇÃO**
---
---

## **1.Importando as Bibliotecas Necessárias**

In [11]:
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
import numpy as np

Para o método HOG teremos 4 dataset, utilizando dos presets:
* tamanho: (128,128) e (256,256)
* método: VGG16 e VGG19
* pooling: MAX e AVG

## **2.Implantando CNN nas imagens**

In [12]:
# Parâmetros
metodo = ['VGG16', 'VGG19']
pooling = ['MAX', 'AVG']
tamanhos = [(128, 128), (256, 256)]

# Função para carregar o modelo uma vez e armazenar as features em um DataFrame
def extrair_features_cnn(m, p, size):
    # Seleciona o modelo baseado em `metodo` e `pooling`
    if m == 'VGG16' and p == 'MAX':
        model = VGG16(weights='imagenet', include_top=False, pooling='max')
    elif m == 'VGG16' and p == 'AVG':
        model = VGG16(weights='imagenet', include_top=False, pooling='avg')
    elif m == 'VGG19' and p == 'MAX':
        model = VGG19(weights='imagenet', include_top=False, pooling='max')
    else:
        model = VGG19(weights='imagenet', include_top=False, pooling='avg')

    data = []

    # Itera sobre todas as imagens no diretório com uma barra de progresso
    for file_path in tqdm(sorted(glob.glob(caminho_das_imagens)), desc=f"Processing {m}_{p}_{size}", unit="image"):
        # Nome do arquivo da imagem
        nome_imagem = os.path.basename(file_path)

        # Rótulo: 1 se a primeira letra do nome do arquivo é minúscula, 0 caso contrário
        label = 1 if nome_imagem[0].islower() else 0

        # Lê e redimensiona a imagem
        imagem = io.imread(file_path)
        imagem_reduzida = resize(imagem, size)  # Anti-aliasing para melhor qualidade

        # Preprocessa a imagem
        x = image.img_to_array(imagem_reduzida)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        # Extrai as features
        features = model.predict(x).flatten()

        # Adiciona o label e as features em uma lista
        data.append([label] + features.tolist())

    # Cria o DataFrame com o label e as features
    colunas = ['label'] + [f'feature_{i}' for i in range(len(features))]
    df_name = f'cnn_{m}_{p}_{size[0]}'  # Nome do DataFrame
    dfs[df_name] = pd.DataFrame(data, columns=colunas)

# Itera sobre as combinações de parâmetros e extrai features para cada uma com barra de progresso externa
for size in tqdm(tamanhos, desc="Sizes", unit="size"):
    for m in tqdm(metodo, desc="Models", unit="model", leave=False):
        for p in tqdm(pooling, desc="Pooling", unit="pooling", leave=False):
            extrair_features_cnn(m, p, size)


Sizes:   0%|          | 0/2 [00:00<?, ?size/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 383ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step




Processing VGG19_AVG_(256, 256): 100%|██████████| 798/798 [03:18<00:00,  4.03image/s]


Sizes: 100%|██████████| 2/2 [16:51<00:00, 505.84s/size]


In [13]:
# Diretório onde os arquivos CSV serão salvos
diretorio_saida = './datasets'
os.makedirs(diretorio_saida, exist_ok=True)

# Salvar cada DataFrame como CSV
for nome_df, df in dfs.items():
    caminho_csv = os.path.join(diretorio_saida, f'{nome_df}.csv')
    df.to_csv(caminho_csv, index=False)
    print(f'Salvou {caminho_csv}')

Salvou ./datasets\hog_128_16.csv
Salvou ./datasets\hog_128_20.csv
Salvou ./datasets\hog_256_16.csv
Salvou ./datasets\hog_256_20.csv
Salvou ./datasets\cnn_VGG16_MAX_128.csv
Salvou ./datasets\cnn_VGG16_AVG_128.csv
Salvou ./datasets\cnn_VGG19_MAX_128.csv
Salvou ./datasets\cnn_VGG19_AVG_128.csv
Salvou ./datasets\cnn_VGG16_MAX_256.csv
Salvou ./datasets\cnn_VGG16_AVG_256.csv
Salvou ./datasets\cnn_VGG19_MAX_256.csv
Salvou ./datasets\cnn_VGG19_AVG_256.csv


In [14]:
for nome_df, df in dfs.items():
  print(nome_df, df.shape)

hog_128_16 (798, 1765)
hog_128_20 (798, 901)
hog_256_16 (798, 8101)
hog_256_20 (798, 4357)
cnn_VGG16_MAX_128 (798, 513)
cnn_VGG16_AVG_128 (798, 513)
cnn_VGG19_MAX_128 (798, 513)
cnn_VGG19_AVG_128 (798, 513)
cnn_VGG16_MAX_256 (798, 513)
cnn_VGG16_AVG_256 (798, 513)
cnn_VGG19_MAX_256 (798, 513)
cnn_VGG19_AVG_256 (798, 513)
